## Libraries Import

In [1]:
import numpy as np
import math
from random import *
import random

## Function

In [2]:
# adjacency matrix
def adjacency_matrix(n):
    a = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if i==j :
                a[i,j] = 0
            elif i<j :
                a[i,j] = randint(0,1)
            else:
                a[i,j] = a[j,i]
    return a 

def rand_func(n, lower, upper):
    return np.random.randint(lower, upper, size = n)

# Variables

In [3]:
np.random.seed(1)
#0. number of clients
num_c = 12

#1. adjacency matrix / connection
A = adjacency_matrix (num_c)
connection = np.sum(A, axis=1)

#2. number of publish
num_pub = rand_func(num_c, 0, 100)

#3. number of subscribe
num_sub = rand_func(num_c, 0, 100)

#4. neighborhood
def neighborhood():
    neighborhood = np.zeros(num_c)
    for i in range(num_c):
        for j in range(num_c):
            if A[i,j]==1 :
                neighborhood[i]+=connection[j]
    return neighborhood

print("1. adjacency matrix = \n",A)
print("connection = \n",connection)
print("2. number of publish = \n", num_pub)
print("3. number of subscribe = \n", num_sub)
print("4. neighborhood =\n", neighborhood())

1. adjacency matrix = 
 [[0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1.]
 [1. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]
 [1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0.]
 [1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1.]
 [0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1.]
 [0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1.]
 [1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0.]]
connection = 
 [5. 6. 7. 5. 8. 9. 5. 4. 3. 6. 7. 5.]
2. number of publish = 
 [37 12 72  9 75  5 79 64 16  1 76 71]
3. number of subscribe = 
 [ 6 25 50 20 18 84 11 28 29 14 50 68]
4. neighborhood =
 [33. 40. 41. 35. 48. 54. 31. 24. 22. 42. 40. 30.]


# ----------------------------------------------------------------------------------------------------------------

# Reinforcement Learning

In [4]:
def rargmax(vector):
    # amax는 최고 값을 가지는 index를 무작위로 선택한다. 
    m = np.amax(vector)
    indices = np.nonzero(vector == m)[0]
    return random.choice(indices)
def sum_pub_minus_sub(v):
    sum_pub =0; sum_sub =0
    seq = sorted(v);
    for i in range(len(num_pub)):
        for j in range(len(seq)):
            if i==seq[j]:
                sum_pub+=num_pub[i]
                sum_sub+=num_sub[i]            
    return sum_pub-sum_sub

def sum_connection(v):
    sum_conn = 0
    seq = sorted(v)
    for i in range(len(connection)):
        for j in range(len(seq)):
            if i==seq[j]:
                sum_conn += connection[i]
    return sum_conn
def sum_neighbor(v):
    sum_ne =0
    neighbor = neighborhood()
    seq = sorted(v)
    for i in range(len(neighbor)):
        for j in range(len(seq)):
            if i==seq[j]:
                sum_ne += neighbor[i]
    return sum_ne
#sum_neighbor([0,1])
#sum_pub_minus_sub([1,2])

In [5]:
state_size = num_c   # number of active peer
action_size = 3 # +1 / 0 / -1
num_episodes = 10
rList=[]  # step 별 reward 저장 
Q = np.zeros([state_size, action_size])
max_reward = [] #lists to contain total rewards 
active_peer_list = []

In [6]:
for i in range(num_episodes):
    #1. 환경을 초기화하고, 첫 state를 가져온다.
    state = 5
    rAll = 0
    done = False 
    max_steps = 8
    step =0
    # Q-table Learning algorithm
    print("-----Episode ", i+1, "-------")
    
    for step in range(max_steps):
        
        reward = 0
        keep_stay = 0 # counting step
        action = rargmax(Q[state,:])
        active_peer_list = []
            
            
        # Get new state and reward from environment
        if action == 0 : 
            keep_stay = 0 
            if state < 0 :
                new_state = state
                reward = -10000000
                done = True
            else :
                new_state = state-1
        elif action ==1:
            new_state = state
            keep_stay +=1
        else:
            keep_stay = 0
            if state > num_c :
                new_sate = state
                reward = -1000000
                done = True
            else :
                new_state = state +1
                    
        if done == True:
            break
        #select #(state) of active peers (print the peers' number)        
        active_peer_list.append(random.randint(0,num_c-1))
        print("step: ", step, " active peer list :", active_peer_list)
            
        #evaluate reward by those peers! 
            #(50-num_active)+(sum_publish-sum_subscribe)*(sum_connection/num_c)*(sum_neighborhood/num_c)
        reward = (num_c/2-new_state)+(sum_pub_minus_sub(active_peer_list))*(sum_connection(active_peer_list)/num_c)*(sum_neighbor(active_peer_list)/num_c)
        reward = round(reward,2)
        print("reward : step은", step,"리워드는,", reward)
        #add list of active peers per 
        # Update Q-table 
        Q[state,action] = reward + np.max(Q[new_state,:])
        print("step",step,"에서","Q는",Q)
        rAll += reward
        state = new_state
        print("new_state는 ", new_state)
    rList.append(rAll)

-----Episode  1 -------
step:  0  active peer list : [0]
reward : step은 0 리워드는, 36.52
step 0 에서 Q는 [[ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.   36.52  0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]]
new_state는  5
step:  1  active peer list : [10]
reward : step은 1 리워드는, 51.56
step 1 에서 Q는 [[ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.   88.08  0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]]
new_state는  5
step:  2  active peer list : [8]
reward : step은 2 리워드는, -4.96
step 2 에서 Q는 [[ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.   83.12  0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0.    0.    0.  ]
 [ 0. 

step 5 에서 Q는 [[   0.      0.      0.  ]
 [   0.      0.      0.  ]
 [   0.      0.      0.  ]
 [   0.      0.      0.  ]
 [   0.      0.      0.  ]
 [   0.    -57.76  687.62]
 [   0.    -22.75  574.69]
 [   0.   -267.62  667.22]
 [ 644.55  -55.39    0.  ]
 [   0.      0.      0.  ]
 [   0.      0.      0.  ]
 [   0.      0.      0.  ]]
new_state는  7
step:  6  active peer list : [0]
reward : step은 6 리워드는, 33.52
step 6 에서 Q는 [[   0.      0.      0.  ]
 [   0.      0.      0.  ]
 [   0.      0.      0.  ]
 [   0.      0.      0.  ]
 [   0.      0.      0.  ]
 [   0.    -57.76  687.62]
 [   0.    -22.75  574.69]
 [   0.   -267.62  678.07]
 [ 644.55  -55.39    0.  ]
 [   0.      0.      0.  ]
 [   0.      0.      0.  ]
 [   0.      0.      0.  ]]
new_state는  8
step:  7  active peer list : [9]
reward : step은 7 리워드는, -23.75
step 7 에서 Q는 [[   0.      0.      0.  ]
 [   0.      0.      0.  ]
 [   0.      0.      0.  ]
 [   0.      0.      0.  ]
 [   0.      0.      0.  ]
 [   0.    -57.76  687.